In [2]:
import scrape_bienici as scrapeBi
import pandas as pd
import get_DistilCamemBERT as camembert

C:\Users\thaim\Documents\python script\analyze_bienetre\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
page = scrapeBi.scrape_pages(search_zone = "paris",start_page=1,end_page=1)

[SYS] : STEP 1 - Getting zone codes for 'paris'...
[SYS] : STEP 2 - Building filters for page 1...
         Filters ready.
[SYS] : STEP 3 - Calling Bienici realEstateAds.json API...
[SYS] : Received 24 ads from Bienici.
<class 'list'>


In [7]:
zc = scrapeBi.get_zoneCode("paris")
zi = zc["zoneIds"]

[SYS] : STEP 1 - Getting zone codes for 'paris'...


In [8]:
zi

['-7444']

In [9]:
page = scrapeBi.scrape_page(zoneIds = zi,page =1)

[SYS] : STEP 2 - Building filters for page 1...
         Filters ready.
[SYS] : STEP 3 - Calling Bienici realEstateAds.json API...
[SYS] : Received 24 ads from Bienici.
<class 'list'>


In [14]:
df = pd.DataFrame(page)

In [15]:
df.columns

Index(['district.code_insee', 'exposition', 'heating', 'description',
       'blurInfo.centroid.lon', 'energyClassification', 'opticalFiberStatus',
       'propertyType', 'departmentCode', 'newProperty', 'hasTerrace', 'photos',
       'city', 'surfaceArea', 'greenhouseGazClassification', 'reference',
       'floor', 'district.libelle', 'blurInfo.bbox',
       'useJuly2021EnergyPerformanceDiagnostic', 'addressKnown',
       'bathroomsQuantity', 'blurInfo.radius', 'transactionType',
       'chargesMethod', 'showerRoomsQuantity', 'energyValue',
       'status.onTheMarket', 'district.insee_code', 'maxEnergyConsumption',
       'greenhouseGazValue', 'blurInfo.centroid.lat', 'rentWithoutCharges',
       'postalCodeForSearchFilters', 'minEnergyConsumption', 'district.name',
       'with3dModel', 'isBienIciExclusive', 'status.highlighted', 'price',
       'blurInfo.type', 'hasElevator', 'descriptionTextLength',
       'toiletQuantity', 'id', 'title', 'energyPerformanceDiagnosticDate',
       '

In [25]:
features = ["postalCode","city","description"]
df[features].dtypes

postalCode     object
city           object
description    object
dtype: object

In [28]:
df["input_text"] = df[features].apply(lambda r : camembert.build_input_text(postalCode = r["postalCode"], city = r["city"],
                                                        description = r["description"],desc_max_chars = 2000),
                                     axis = 1)

In [30]:
tokenizer, model = camembert.load_model("models/location_flagger_v5")


In [33]:
df["falseLocation"] = df["input_text"].apply(lambda x : camembert.predict(input_model = model, tokenizer = tokenizer, input_text = x))

In [34]:
df["falseLocation"]

0         (1, 0.9999608993530273)
1     (0, 5.5718683142913505e-06)
2     (0, 3.2588022804702632e-06)
3          (1, 0.995126485824585)
4         (1, 0.9796792268753052)
5     (0, 2.5610675038478803e-06)
6      (0, 9.283096915169153e-06)
7      (0, 3.467970373094431e-06)
8      (0, 2.483588332324871e-06)
9         (1, 0.9988985061645508)
10        (1, 0.9999716281890869)
11        (1, 0.9997738003730774)
12     (0, 5.000484634365421e-06)
13     (0, 5.376320132199908e-06)
14    (0, 3.6740100313181756e-06)
15     (0, 3.406005816941615e-06)
16    (0, 3.7480745049833786e-06)
17     (0, 3.660157972262823e-06)
18     (0, 3.159653942930163e-06)
19     (0, 2.183906190111884e-06)
20    (0, 1.8392480569673353e-06)
21        (0, 0.5264368653297424)
22    (0, 3.5652310543810017e-06)
23        (1, 0.9999351501464844)
Name: falseLocation, dtype: object

"[POSTAL] 75015\n[CITY] Paris 15e\n[DESC]\nSitué rue Cambronne à proximité de toutes les commodités, un appartement meublé au 2ème étage totalement refait à neuf.\nIl se compose d'une entrée avec un placard de rangement, un cuisine ouverte et équipée donnant sur la salle à manger et le séjour, une chambre donnant sur cour avec penderie, une salle d'eau et un wc.\nCet appartement meublé de 45m2 comprend tout le charme de l'ancien (parquet, moulures, cheminées)\n\nDisponible immédiatement.\n\nZone soumise à encadrement des loyers\nLoyer de référence majoré (loyer de base à ne pas dépasser) : 1645€\nLoyer de base : 1675€ par mois\nProvision pour charges : 96€\nSoit 1771€ charges comprises\n\nDépot de garantie: 3290€"

In [3]:
df = pd.read_csv("ressources/data/bienIci_2458.csv")

In [10]:
def analyze_duplicate_ids(df: pd.DataFrame, id_col: str = "id", ignore_cols: list[str] | None = None):
    if ignore_cols is None:
        ignore_cols = []

    # work only on duplicated ids
    dup_df = df[df[id_col].duplicated(keep=False)].copy()

    # columns to compare (ignore id + any noisy columns you choose)
    compare_cols = [c for c in dup_df.columns if c not in ([id_col] + ignore_cols)]

    # normalize for reliable equality: treat NaN/NA consistently as a token
    norm = dup_df[compare_cols].astype("string").fillna("__NA__")

    # for each id group, count how many distinct "rows" exist across compare_cols
    distinct_count = (
        norm.groupby(dup_df[id_col])
            .apply(lambda g: g.drop_duplicates().shape[0])
            .rename("distinct_rows")
            .reset_index()
    )

    # split into "same content" vs "different content"
    same_ids = distinct_count.loc[distinct_count["distinct_rows"] == 1, id_col]
    diff_ids = distinct_count.loc[distinct_count["distinct_rows"] > 1, id_col]

    dup_same = dup_df[dup_df[id_col].isin(same_ids)].copy()
    dup_diff = dup_df[dup_df[id_col].isin(diff_ids)].copy()

    return distinct_count.sort_values("distinct_rows", ascending=False), dup_same, dup_diff

In [14]:
df.drop_duplicates()

,transactionType,rentWithoutCharges,charges,energyPerformanceDiagnosticDate,title,heating,district.name,hasTerrace,greenhouseGazClassification,greenhouseGazValue,...,bedroomsQuantity,descriptionTextLength,postalCodeForSearchFilters,price,city,blurInfo.type,status.onTheMarket,district.libelle,blurInfo.centroid.lon,hasElevator
0,rent,1771.0,NaN,2023-09-19,Exclusivité- Appartement meublé- Paris XV,électricité individuel,Paris 15e Arrondissement - Cambronne - Garibaldi,NaN,B,6.0,...,1.0,667,75015,1771.0,Paris 15e,disk,True,Cambronne - Garibaldi,2.303488,NaN
1,rent,1500.0,450.0,2024-06-25,2 pièces avec terrasse - Paris 15,radiateur gaz collectif,Paris 15e Arrondissement - Alleray - Procession,True,C,29.0,...,1.0,474,75015,1950.0,Paris 15e,disk,True,Alleray - Procession,2.304538,True
2,rent,13000.0,NaN,2025-07-06,Appartement exceptionnel- Paris VIII,NaN,Paris 8e Arrondissement - Hoche Friedland,True,D,33.0,...,4.0,850,75008,13000.0,Paris 8e,disk,True,Hoche Friedland,2.299560,NaN
3,rent,3800.0,NaN,2025-12-02,Appartement meublé - Paris 14ème,NaN,Paris 14e Arrondissement - Montsouris - Dareau,NaN,D,34.0,...,2.0,701,75014,3800.0,Paris 14e,disk,True,Montsouris - Dareau,2.335510,NaN
4,rent,866.0,NaN,2023-02-20,STUDIO-ILES SAINT LOUIS,NaN,Paris 4e Arrondissement - Les Iles,NaN,B,11.0,...,0.0,311,75004,866.0,Paris 4e,disk,True,Les Iles,2.354162,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1602,rent,4500.0,NaN,NaN,Location Appartement Meublé PARIS 06,gaz individuel,Paris 6e Arrondissement,False,NaN,NaN,...,3.0,645,75006,4500.0,Paris 6e,cityOrArrondissement,True,Paris 6,2.332951,True
1603,rent,2710.0,NaN,NaN,Location Appartement Meublé PARIS 75017,NaN,Paris 17e Arrondissement,False,NaN,NaN,...,2.0,692,75017,2710.0,Paris 17e,cityOrArrondissement,True,Paris 17,2.322364,False
1604,rent,3300.0,NaN,NaN,Location Appartement Meublé PARIS 7,NaN,Paris 7e Arrondissement,False,NaN,48.0,...,2.0,345,75007,3300.0,Paris 7e,cityOrArrondissement,True,Paris 7,2.320195,True
1605,rent,NaN,NaN,2023-02-27,Appartement deux pièces 35 M² PARIS 12 - Métro...,électrique individuel,Paris 12e Arrondissement - Vallée de Fecamp,NaN,C,12.0,...,1.0,499,75012,NaN,Paris 12e,disk,True,Vallée de Fecamp,2.397793,False


In [15]:
import osm

In [19]:
pois = osm.get_pois(lat = 48.88446954640768, lon = 2.335555934742604)

In [18]:
copy_pois = pois 

In [31]:
copy_pois["poi_categories"].keys()
copy_pois.keys()

dict_keys(['input', 'poi_categories', 'nearest'])

In [29]:
copy_pois["poi_categories"]["restaurants_like"].keys()

dict_keys(['count', 'nearest'])

In [27]:
restaurants_df = pd.DataFrame(copy_pois["poi_categories"]["restaurants_like"]["nearest"])

In [28]:
restaurants_df.head()

,name,dist_km,lat,lon,address,gmaps_url
0,Chez Ammad,0.067969,48.884601,2.336461,None,https://www.google.com/maps/search/?api=1&quer...
1,Chez Ammad,0.068185,48.884605,2.336463,"Rue Véron, 75009 Paris",https://www.google.com/maps/search/?api=1&quer...
2,Armelle et Henri,0.069386,48.885075,2.335786,"Rue Audran, 75018",https://www.google.com/maps/search/?api=1&quer...
3,Shunei,0.074957,48.885125,2.335792,None,https://www.google.com/maps/search/?api=1&quer...
4,Piccolo Caratello,0.080662,48.885134,2.335996,"Rue Audran, 75018 Paris",https://www.google.com/maps/search/?api=1&quer...
